#  Imports verses from Tanakh into a LaTeX-friendly format

In [ ]:
#this parses the formatting notes in the 3rd column of the spreadsheet

In [54]:
format = {
   "//{{פפ}}//":"\p",
   "{{סס}}":"\s"
}

In [62]:
def format_parser(format_in):
    blank = '__'
    if format_in == blank:
        return ''
    elif format_in in format:
        return format[format_in]
    else:
        return format_in

In [ ]:
#this looks for a note about the aliya in the 4th column of the spreadsheet.
#maybe functionality for sedarim will be added over time.

In [92]:
def aliya_parser(aliya_in):
    aliya = 'עלייה'
    if aliya in aliya_in:
        aliya_start=0
        while aliya_start<len(aliya_in) and aliya_in[aliya_start]!='=':
            aliya_start=aliya_start+1
        aliya_end = aliya_start+1
        while aliya_end<len(aliya_in) and aliya_in[aliya_end]!='}':
            aliya_end = aliya_end+1
        return aliya_in[aliya_start+1:aliya_end]
    else:
        return ''

In [ ]:
#this parses notes embedded in the tanakh-text.

In [96]:
def text_parser(pasuk_in):
    #needs support for kri ukhtiv
    #needs support for footnotes
    #needs support for special characters
    return pasuk_in

In [97]:
#this pulls verses from the Torah and runs them through the above parsers

In [98]:
import csv
torah = 'torah.csv'
book_in = 'ספר שמות'
perek_start = 'א'
perek_end = 'א'
pasuk_start = 'א'
pasuk_end = 'ח'
output = ''
with open(torah, 'r', encoding='utf-8') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    in_range = False
    for row in csvReader:
        book_perek = row[0].split('/')
        pasuk = row[1]
        if book_in == book_perek[0] and perek_start==book_perek[1] and pasuk_start==pasuk:
            in_range = True
        
        if in_range==True:
            output = output+format_parser(row[2])+aliya_parser(row[3])+text_parser(row[4])
            print(format_parser(row[2]),'\n',aliya_parser(row[3]),'\n',row[4])
        
        if book_in == book_perek[0] and perek_end==book_perek[1] and pasuk_end==pasuk:
            break

{{מ:ספר חדש|שמות}} 
 א|עלייה='''שמות''' 
 וְאֵ֗לֶּה שְׁמוֹת֙ בְּנֵ֣י יִשְׂרָאֵ֔ל הַבָּאִ֖ים מִצְרָ֑יְמָה אֵ֣ת יַעֲקֹ֔ב אִ֥ישׁ וּבֵית֖וֹ בָּֽאוּ׃
 
  
 רְאוּבֵ֣ן שִׁמְע֔וֹן לֵוִ֖י וִיהוּדָֽה׃
 
  
 יִשָּׂשכָ֥ר זְבוּלֻ֖ן {{נוסח|וּבִנְיָמִֽן|=שש1 ובדפוסים|ל=וּבְנְיָמִֽן (בי"ת בשתי נקודות שוא!)|הערות דותן והמקליד}}׃
 
  
 דָּ֥ן וְנַפְתָּלִ֖י גָּ֥ד וְאָשֵֽׁר׃
 
  
 {{נוסח|וַֽיְהִ֗י|ל=וַֽיְהִ֗י בגעיה ימנית}} כׇּל־נֶ֛פֶשׁ יֹצְאֵ֥י יֶֽרֶךְ־יַעֲקֹ֖ב שִׁבְעִ֣ים נָ֑פֶשׁ וְיוֹסֵ֖ף הָיָ֥ה בְמִצְרָֽיִם׃
 
  
 וַיָּ֤מׇת יוֹסֵף֙ וְכׇל־אֶחָ֔יו וְכֹ֖ל הַדּ֥וֹר הַהֽוּא׃
 
  
 וּבְנֵ֣י יִשְׂרָאֵ֗ל פָּר֧וּ וַֽיִּשְׁרְצ֛וּ וַיִּרְבּ֥וּ וַיַּֽעַצְמ֖וּ בִּמְאֹ֣ד מְאֹ֑ד וַתִּמָּלֵ֥א הָאָ֖רֶץ אֹתָֽם׃
\p 
 [לוי] 
 וַיָּ֥קׇם מֶֽלֶךְ־חָדָ֖שׁ עַל־מִצְרָ֑יִם אֲשֶׁ֥ר לֹֽא־יָדַ֖ע אֶת־יוֹסֵֽף׃


In [95]:
output

'{{מ:ספר חדש|שמות}}א|עלייה=\'\'\'שמות\'\'\'וְאֵ֗לֶּה שְׁמוֹת֙ בְּנֵ֣י יִשְׂרָאֵ֔ל הַבָּאִ֖ים מִצְרָ֑יְמָה אֵ֣ת יַעֲקֹ֔ב אִ֥ישׁ וּבֵית֖וֹ בָּֽאוּ׃רְאוּבֵ֣ן שִׁמְע֔וֹן לֵוִ֖י וִיהוּדָֽה׃יִשָּׂשכָ֥ר זְבוּלֻ֖ן {{נוסח|וּבִנְיָמִֽן|=שש1 ובדפוסים|ל=וּבְנְיָמִֽן (בי"ת בשתי נקודות שוא!)|הערות דותן והמקליד}}׃דָּ֥ן וְנַפְתָּלִ֖י גָּ֥ד וְאָשֵֽׁר׃{{נוסח|וַֽיְהִ֗י|ל=וַֽיְהִ֗י בגעיה ימנית}} כׇּל־נֶ֛פֶשׁ יֹצְאֵ֥י יֶֽרֶךְ־יַעֲקֹ֖ב שִׁבְעִ֣ים נָ֑פֶשׁ וְיוֹסֵ֖ף הָיָ֥ה בְמִצְרָֽיִם׃וַיָּ֤מׇת יוֹסֵף֙ וְכׇל־אֶחָ֔יו וְכֹ֖ל הַדּ֥וֹר הַהֽוּא׃וּבְנֵ֣י יִשְׂרָאֵ֗ל פָּר֧וּ וַֽיִּשְׁרְצ֛וּ וַיִּרְבּ֥וּ וַיַּֽעַצְמ֖וּ בִּמְאֹ֣ד מְאֹ֑ד וַתִּמָּלֵ֥א הָאָ֖רֶץ אֹתָֽם׃\\p[לוי]וַיָּ֥קׇם מֶֽלֶךְ־חָדָ֖שׁ עַל־מִצְרָ֑יִם אֲשֶׁ֥ר לֹֽא־יָדַ֖ע אֶת־יוֹסֵֽף׃'

## Imports verses from megillot

In [ ]:
import csv
megillot = 'chamesh megillot.csv'

## Imports verses from Navi

In [ ]:
navi1 = 'neviim rishonim.csv'
navi2 = 'neviim achronim.csv'
import csv